### Installing dependencies:

In [1]:
%pip install accelerate
%pip install nltk
%pip install mosestokenizer
!pip install evaluate
!pip install rouge_score
!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!pip install --editable /kaggle/working/IndicTransTokenizer
!pip install torch
!pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49170 sha256=15e420c923ff4fae884e8b7bdf95d0360d6e6fc9c33b6f7313c41b20de4e5d9a
  Stored in directory: /root/.cache/pip/wheels/80/d8/15/4c5ebbe883513f003cb055a0369c77c9df857023a706f39e70
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3338 sha256=1f89c32cab90e1902e4c6d4b836736215bb387084fb36fe0a35403086b95d081
  Stored in directory: /root/.cache/pip/wheels/e1/af/b1/99b57a06dda78fdcee86d2e22c64743f3b8df8f31cfc04baf7
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6147 sha256=0a0c7f26555ca3a78b5d8f8985a20c362fcd42dbf314c84d3a72b541c4388539
  Stored in directory: /root/.cache/pip/wheels/05/ee/10

### Reading dataset

In [2]:
import numpy as np 
import pandas as pd 
import os
sentences=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        print("reading..",path)
        with open(path,'r',encoding='utf-8') as file:
            text = file.readlines()
            text = [x.strip().replace('\u200d', '') for x in text]
        temp = []
        for line in text:
            temp.append(line)
        sentences.append(temp)

reading.. /kaggle/input/test-data/test.mr
reading.. /kaggle/input/test-data/test.hi
reading.. /kaggle/input/test-data/test.en


In [3]:
import random
indexes = random.sample(range(0,len(sentences[0])),1000)
mar_sentences  = [sentences[0][i] for i in indexes]
hi_sentences = [sentences[1][i] for i in indexes]
eng_sentences = [sentences[2][i] for i in indexes]

In [4]:
print(hi_sentences[:5])
print(eng_sentences[:5])
print(mar_sentences[:5])

['उन्होंने कहा कि श्री इनामदार ने इस सिद्धांत को अपने जीवन में आत्मसात किया था और उनका जीवन प्रेरणा का एक स्रोत है।', 'प्रधानमंत्री ने शिष्टमंडल के सदस्यों द्वारा पूछे गए विभिन्न प्रश्नों का विस्तार से जवाब दिया।', 'हमारे यूपीआई अथवा यूनाइटेड पेमेंट्स इंटरफेस सिस्टम के साथ-साथ भीम एप तथा रुपे कार्ड भी भारत में डिजिटल अर्थव्यवस्था की वास्तविक संभावनाओं को दर्शा रहे हैं।', 'आपदा पीड़ित लोगों को देखना पीड़ादायक होता है।', 'यह दोनों देशों की साझा सांस्कृतिक विरासत को ध्यान में रखते हुए दोनों देशों के लिए अत्यंत महत्वपूर्ण सिद्ध होगा।']
['He said that Shri Inamdar had imbibed this principle, and his life is a source of inspiration.', 'The Prime Minister responded at length to various questions raised by the delegation members.', 'Our United Payments Interface system or UPI along with BHIM App and RuPay Card has shown the true potential of digital economy in India.', 'It was distressing to see the suffering of the affected people.', 'This will be of immense importance to both countries consi

### writing dataset (optional)

In [5]:
with open("hi_sentences.txt", "w",encoding='utf-8') as file:
    for line in hi_sentences:
        file.write(line+'\n')
with open("eng_sentences.txt", "w",encoding='utf-8') as file:
    for line in eng_sentences:
        file.write(line+'\n')
with open("mar_sentences.txt", "w",encoding='utf-8') as file:
    for line in mar_sentences:
        file.write(line+'\n')

#### If want to read from these files then use this function

In [6]:
# def openFiles(filepath):
#     with open(filepath, 'r',encoding='utf-8') as file:
#         data = file.readlines()
#         data = [line.strip().replace('\u200d','') for line in data]
#         return data
# hi_sentences = openFiles("/kaggle/working/hi_sentences.txt")
# eng_sentences = openFiles("/kaggle/working/eng_sentences.txt")
# mar_sentences = openFiles("/kaggle/working/mar_sentences.txt")

In [7]:
# print(hi_sentences[:5])
# print(eng_sentences[:5])
# print(mar_sentences[:5])

### Functions to compute required matrices->

In [8]:
import evaluate
from nltk.tokenize import word_tokenize
def compute_matrice(predictions,references):
    bleu = evaluate.load("bleu")
    rogue = evaluate.load('rouge')
    bleu_score = bleu.compute(predictions=predictions, references=references)
    rogue_score = rogue.compute(predictions=predictions, references=references)
    print(bleu_score)
    print("="*15)
    print(rogue_score)

2024-04-17 04:16:09.915822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 04:16:09.915917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 04:16:10.048245: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### NLLB

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

### To use GPU if avail, otherwise CPU will be used by default

In [10]:
%%time
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model=model.to(device)
# (may give error if all tensors are not shifted to GPU)

Using device: cuda
CPU times: user 610 ms, sys: 148 ms, total: 758 ms
Wall time: 801 ms


#### Function for translation from src_lang to tgt_lang

In [11]:
%%time
def translate_sentence(sentence, src_lang="eng_Latn", tgt_lang="hin_Deva"):
    inputs = tokenizer(sentence, return_tensors="pt")
    inputs = inputs.to(device)
    #  tokenize the input sentence
    translated_ids = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
    # Decode the tokens
    translated_sentence = tokenizer.batch_decode(translated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return translated_sentence[0]

def write_file(filename,sentences):
    with open(filename,"w",encoding='utf-8') as file:
        for sentence in sentences:
            file.write(sentence+'\n')

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [12]:
%%time
# ===================English to Hindi==========================
en_2_hi_sentences_nllb = []
for sentence in eng_sentences:
    en_2_hi_sentences_nllb.append(translate_sentence(sentence,src_lang="eng_Latn", tgt_lang="hin_Deva"))
print(en_2_hi_sentences_nllb[:5])
print("\n Blue and rogue Scores for eng to hi translation by NLLB:")
print("="*15)

# Calculating required matrices where en_2_hi_sentences_nllb is translated hi sentence by nllb and hi_sentences are original sentences
compute_matrice(en_2_hi_sentences_nllb,hi_sentences)

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


['उन्होंने कहा कि श्री इनामदार ने इस सिद्धांत को अपनाया है और उनका जीवन प्रेरणा का स्रोत है।', 'प्रधानमंत्री ने प्रतिनिधिमंडल के सदस्यों द्वारा उठाए गए विभिन्न सवालों के विस्तार से जवाब दिए।', 'हमारी यूनाइटेड पेमेंट्स इंटरफेस प्रणाली या यूपीआई भीम ऐप और रूपई कार्ड के साथ भारत में डिजिटल अर्थव्यवस्था की वास्तविक क्षमता को दिखा रही है।', 'पीड़ित लोगों के दुख को देखना दुखद था।', 'दोनों देशों के लिए यह बहुत महत्वपूर्ण होगा क्योंकि दोनों देश अपनी साझा सांस्कृतिक विरासत को देखते हैं।']

 Blue and rogue Scores for eng to hi translation by NLLB:


{'bleu': 0.3214126874851807, 'precisions': [0.6341702741702742, 0.39822358346094944, 0.2699334464309017, 0.18671040692398966], 'brevity_penalty': 0.9569152934477807, 'length_ratio': 0.9578173374613003, 'translation_length': 17325, 'reference_length': 18088}
{'rouge1': 0.12157619047619046, 'rouge2': 0.029000000000000005, 'rougeL': 0.12095952380952382, 'rougeLsum': 0.12096190476190477}
CPU times: user 7min 28s, sys: 3.81 s, total: 7min 32s
Wall time: 7min 34s


In [13]:
%%time
# ===================Hindi to English==========================
hi_2_en_sentences_nllb = []
for sentence in hi_sentences:
    hi_2_en_sentences_nllb.append(translate_sentence(sentence,src_lang="hin_Deva", tgt_lang="eng_Latn"))
print(hi_2_en_sentences_nllb[:5])
print("\n Blue and rogue Scores for hi to eng translation by NLLB:")
print("="*15)

# Calculating required matrices where hi_2_en_sentences_nllb is translated eng sentence by nllb and eng_sentences are original sentences
compute_matrice(hi_2_en_sentences_nllb,eng_sentences)

['He said that Shri Inamdar had incorporated this principle into his life and his life is a source of inspiration.', 'The Prime Minister answered various questions asked by the delegation members in detail.', 'Along with our UPI or United Payments Interface System, BHIM App and Rupee Card are also showing the real potential of the digital economy in India.', 'It is painful to see people suffering from disasters.', 'This will prove to be extremely important for both countries, taking into account the shared cultural heritage of both countries.']

 Blue and rogue Scores for hi to eng translation by NLLB:
{'bleu': 0.3635644843798687, 'precisions': [0.6629207169747711, 0.42501637197118536, 0.29600557067797684, 0.20948776861738072], 'brevity_penalty': 1.0, 'length_ratio': 1.0045154371507592, 'translation_length': 17797, 'reference_length': 17717}
{'rouge1': 0.6820677456843711, 'rouge2': 0.4591287514619859, 'rougeL': 0.6360717354972417, 'rougeLsum': 0.6358223571072656}
CPU times: user 7min 1

In [14]:
%%time
# ===================Marathi to Hindi==========================
mar_2_hi_sentences_nllb = []
for sentence in mar_sentences:
    mar_2_hi_sentences_nllb.append(translate_sentence(sentence,src_lang="mar_Deva", tgt_lang="hin_Deva"))
print(mar_2_hi_sentences_nllb[:5])
print("\n Blue and rogue Scores for mar to hi translation by NLLB:")
print("="*15)

# Calculating required matrices where mar_2_hi_sentences_nllb is translated mar sentence by nllb and hi_sentences are original sentences
compute_matrice(mar_2_hi_sentences_nllb,hi_sentences)

['उन्होंने कहा कि पुरस्कारदाताओं ने इस तत्व को अपनाया है और उनका जीवन प्रेरणा का स्रोत है।', 'प्रधानमंत्री ने प्रतिनिधिमंडल के सदस्यों द्वारा उपस्थित विभिन्न प्रश्नों का विस्तृत समाधान किया।', 'हमारे यूनाइटेड पेमेंट इंटरफेस सिस्टम या यूपीआई, भीम ऐप और रुपे कार्ड ने भारत की डिजिटल अर्थव्यवस्था की खरी क्षमता दिखाई है।', 'या भूकंप की झलक से प्रभावित लोगों की हालत को देखना बहुत दुखद होता है।', 'दोनों देशों की सांस्कृतिक परंपराओं को देखते हुए यह समझौता बहुत महत्वपूर्ण है।']

 Blue and rogue Scores for mar to hi translation by NLLB:
{'bleu': 0.21083193639889194, 'precisions': [0.5382363274388101, 0.28790894389849486, 0.1646770128664279, 0.09808935293699837], 'brevity_penalty': 0.9425744086858198, 'length_ratio': 0.9441618752764264, 'translation_length': 17078, 'reference_length': 18088}
{'rouge1': 0.1092404761904762, 'rouge2': 0.022, 'rougeL': 0.10882857142857144, 'rougeLsum': 0.10905952380952383}
CPU times: user 7min 24s, sys: 3.66 s, total: 7min 27s
Wall time: 7min 29s


In [15]:
%%time
# ===================Hindi to Marathi==========================
hi_2_mar_sentences_nllb = []
for sentence in hi_sentences:
    hi_2_mar_sentences_nllb.append(translate_sentence(sentence,src_lang="hin_Deva", tgt_lang="mar_Deva"))
print(hi_2_mar_sentences_nllb[:5])
print("\n Blue and rogue Scores for hi to mar translation by NLLB:")
print("="*15)

# Calculating required matrices where mar_2_hi_sentences_nllb is translated mar sentence by nllb and hi_sentences are original sentences
compute_matrice(hi_2_mar_sentences_nllb,mar_sentences)

['श्री इनामदार यांनी हा सिद्धांत आपल्या जीवनात आत्मसात केला आणि त्यांचे जीवन प्रेरणादायी आहे, असे ते म्हणाले.', 'शिष्टमंडळातील सदस्यांनी उपस्थित केलेल्या विविध प्रश्नांची पंतप्रधानांनी उत्तरे दिली.', 'आमच्या यूपीआय किंवा युनायटेड पेमेंट्स इंटरफेस सिस्टमसोबतच भीम अॅप आणि रुपे कार्ड देखील भारतात डिजिटल अर्थव्यवस्थेच्या वास्तविक शक्यता दर्शवत आहेत.', 'आपत्ती पीडित लोकांना पाहणे वेदनादायक असते.', 'दोन्ही देशांच्या सामायिक सांस्कृतिक वारसा लक्षात घेऊन दोन्ही देशांसाठी हे अत्यंत महत्त्वाचे ठरेल.']

 Blue and rogue Scores for hi to mar translation by NLLB:
{'bleu': 0.15616568865017624, 'precisions': [0.47098467650397274, 0.21747098350641417, 0.10953128874927669, 0.058733447437167824], 'brevity_penalty': 0.9747150548278938, 'length_ratio': 0.975029397523691, 'translation_length': 14096, 'reference_length': 14457}
{'rouge1': 0.08963333333333333, 'rouge2': 0.015666666666666666, 'rougeL': 0.08828333333333334, 'rougeLsum': 0.08831666666666668}
CPU times: user 7min 47s, sys: 3.11 s, total: 7min 50s

### uncomment if you want to dump these files

In [16]:
write_file("en_2_hi_sentences_nllb.txt",en_2_hi_sentences_nllb)
write_file("hi_2_en_sentences_nllb.txt",hi_2_en_sentences_nllb)
write_file("mar_2_hi_sentences_nllb.txt",hi_2_en_sentences_nllb)
write_file("hi_2_mar_sentences_nllb.txt",hi_2_en_sentences_nllb)

## IndicTrans Model

In [17]:
%%time
import sys
import torch
from IndicTransTokenizer import IndicTransTokenizer, IndicProcessor
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from mosestokenizer import MosesSentenceSplitter
from nltk import sent_tokenize
from indicnlp.tokenize.sentence_tokenize import sentence_split, DELIM_PAT_NO_DANDA


en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"  
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-dist-200M"  
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M" 
BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

if len(sys.argv) > 1:
    quantization = sys.argv[1]
else:
    quantization = ""

def split_sentences(input_text, lang):
    if lang == "eng_Latn":
        input_sentences = sent_tokenize(input_text)
        with MosesSentenceSplitter(flores_codes[lang]) as splitter:
            sents_moses = splitter([input_text])
        sents_nltk = sent_tokenize(input_text)
        if len(sents_nltk) < len(sents_moses):
            input_sentences = sents_nltk
        else:
            input_sentences = sents_moses
        input_sentences = [sent.replace("\xad", "") for sent in input_sentences]
    else:
        input_sentences = sentence_split(input_text, lang=flores_codes[lang], delim_pat=DELIM_PAT_NO_DANDA)
    return input_sentences


def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        # model.half() gpu

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decodeing the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocessing the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

CPU times: user 485 ms, sys: 13 ms, total: 498 ms
Wall time: 527 ms


#### Intializing models and tokenizers

In [18]:
ip = IndicProcessor(inference=True)

en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", quantization)
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, "indic-indic", quantization)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [19]:
%%time
# ============================English to Hindi====================================
en_2_hindi_IndicTrans = []
en_2_hindi_IndicTrans = batch_translate(eng_sentences, "eng_Latn", "hin_Deva", en_indic_model, en_indic_tokenizer, ip)
print(en_2_hindi_IndicTrans[:5])
print("\n Blue and rogue Scores for eng to hi translation by IndicTranse:")
print("="*15)

# Calculating required matrices where en_2_hindi_IndicTrans is translated hi sentence by IndicTranse and hi_sentences are original sentences
compute_matrice(en_2_hindi_IndicTrans,hi_sentences)

['उन्होंने कहा कि श्री इनामदार ने इस सिद्धांत को अपनाया था और उनका जीवन प्रेरणा का स्रोत है।', 'प्रधानमंत्री ने प्रतिनिधिमंडल के सदस्यों द्वारा उठाए गए विभिन्न प्रश्नों का विस्तार से उत्तर दिया।', 'भीम ऐप और रुपे कार्ड के साथ हमारी यूनाइटेड पेमेंट्स इंटरफेस सिस्टम या यू. पी. आई. ने भारत में डिजिटल अर्थव्यवस्था की वास्तविक क्षमता को दिखाया है।', 'प्रभावित लोगों की पीड़ा को देखना दुखद था।', 'दोनों देशों की साझा सांस्कृतिक विरासत को देखते हुए यह दोनों देशों के लिए बहुत महत्वपूर्ण होगा।']

 Blue and rogue Scores for eng to hi translation by IndicTranse:
{'bleu': 0.3620301184204696, 'precisions': [0.6457446220206824, 0.42451560030439617, 0.29638149714001494, 0.21166721909181305], 'brevity_penalty': 0.999723535430696, 'length_ratio': 0.9997235736399823, 'translation_length': 18083, 'reference_length': 18088}
{'rouge1': 0.11841648351648354, 'rouge2': 0.02362943722943723, 'rougeL': 0.11796355311355312, 'rougeLsum': 0.1177423076923077}
CPU times: user 3min 6s, sys: 5.97 s, total: 3min 12s
Wall 

In [20]:
# ============================Hindi to English====================================
hindi_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-dist-200M"
hi_2_en_IndicTrans = batch_translate(hi_sentences, "hin_Deva", "eng_Latn", indic_en_model, indic_en_tokenizer, ip)
print(hi_2_en_IndicTrans[:5])
print("\n Blue and rogue Scores for hi to eng translation by IndicTranse:")
print("="*15)

# Calculating required matrices where hi_2_en_IndicTrans is translated English sentence by IndicTranse and eng_sentences are original sentences
compute_matrice(hi_2_en_IndicTrans, eng_sentences)

['He said that Shri Inamdar had imbibed this principle in his life and his life is a source of inspiration.', 'The Prime Minister answered various questions posed by the members of the delegation in detail.', 'Our UPI or United Payments Interface system, as well as the BHIM app and RuPay cards, are also showing the real potential of the digital economy in India.', 'It is painful to see people affected by a disaster.', 'This will prove to be extremely important for both the countries considering the shared cultural heritage of both the countries.']

 Blue and rogue Scores for hi to eng translation by IndicTranse:
{'bleu': 0.4107323138301978, 'precisions': [0.6907308377896613, 0.4682633317602643, 0.3431956371842287, 0.25638625117025543], 'brevity_penalty': 1.0, 'length_ratio': 1.013264096630355, 'translation_length': 17952, 'reference_length': 17717}
{'rouge1': 0.7220278572365707, 'rouge2': 0.5162517617918585, 'rougeL': 0.6765146861920868, 'rougeLsum': 0.6762723758196599}


In [21]:
# ============================Marathi to Hindi====================================
marathi_hindi_ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M"
mar_2_hindi_IndicTrans = batch_translate(mar_sentences, "mar_Deva", "hin_Deva", indic_indic_model, indic_indic_tokenizer, ip)
print(mar_2_hindi_IndicTrans[:5])
print("\n Blue and rogue Scores for mr to hi translation by IndicTranse:")
print("="*15)

# Calculating required matrices where mar_2_hindi_IndicTrans is translated hi sentence by IndicTranse and hi_sentences are original sentences
compute_matrice(mar_2_hindi_IndicTrans, hi_sentences)

['उन्होंने कहा कि इनामदारों ने इस सिद्धांत को अपनाया और उनका जीवन प्रेरणा का स्रोत है।', 'प्रधानमंत्री ने प्रतिनिधिमंडल के सदस्यों द्वारा उठाए गए विभिन्न प्रश्नों का विस्तार से समाधान किया।', 'हमारी संयुक्त भुगतान इंटरफेस प्रणाली या यूपीआई, भीम ऐप और रुपे कार्ड ने भारत में डिजिटल अर्थव्यवस्था की वास्तविक क्षमता को दिखाया है।', 'भूकंप से प्रभावित लोगों की पीड़ा को देखना बहुत दुखद था।', 'दोनों देशों की सांस्कृतिक परंपराओं को देखते हुए यह समझौता ज्ञापन बहुत महत्वपूर्ण है।']

 Blue and rogue Scores for mr to hi translation by IndicTranse:
{'bleu': 0.24581565656953122, 'precisions': [0.5512290502793296, 0.30940828402366866, 0.1882900446512798, 0.11857468796134747], 'brevity_penalty': 0.9895521684518365, 'length_ratio': 0.9896063688633349, 'translation_length': 17900, 'reference_length': 18088}
{'rouge1': 0.11039285714285715, 'rouge2': 0.022333333333333334, 'rougeL': 0.11047619047619048, 'rougeLsum': 0.11027619047619047}


In [22]:
# ============================Hindi to Marathi====================================
hindi_marathi_ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M"
hi_2_marathi_IndicTrans = batch_translate(hi_sentences, "hin_Deva", "mar_Deva", indic_indic_model, indic_indic_tokenizer, ip)
print(hi_2_marathi_IndicTrans[:5])
print("\n Blue and rogue Scores for hi to mr translation by IndicTranse:")
print("="*15)

# Calculating required matrices where hi_2_marathi_IndicTrans is translated Marathi sentence by IndicTranse and mar_sentences are original sentences
compute_matrice(hi_2_marathi_IndicTrans, mar_sentences)

['ते म्हणाले की श्री. इनामदार यांनी त्यांच्या जीवनात हा सिद्धांत आत्मसात केला होता आणि त्यांचे जीवन हा प्रेरणेचा स्रोत आहे.', 'शिष्टमंडळाच्या सदस्यांनी विचारलेल्या विविध प्रश्नांना पंतप्रधानांनी सविस्तर उत्तर दिले.', 'आमची यू. पी. आय. किंवा युनायटेड पेमेंट्स इंटरफेस प्रणाली तसेच भीम अॅप आणि रुपे कार्ड देखील भारतातील डिजिटल अर्थव्यवस्थेची खरी शक्यता दर्शवित आहेत.', 'आपत्तीग्रस्त लोकांना पाहणे वेदनादायक आहे.', 'दोन्ही देशांचा सामायिक सांस्कृतिक वारसा लक्षात घेता हे दोन्ही देशांसाठी अत्यंत महत्त्वाचे ठरेल.']

 Blue and rogue Scores for hi to mr translation by IndicTranse:
{'bleu': 0.15953046903355353, 'precisions': [0.4663871229541458, 0.21782968228637747, 0.10909517478689294, 0.05843935538592027], 'brevity_penalty': 1.0, 'length_ratio': 1.0227571418689907, 'translation_length': 14786, 'reference_length': 14457}
{'rouge1': 0.09161666666666668, 'rouge2': 0.01873333333333333, 'rougeL': 0.09115238095238096, 'rougeLsum': 0.09129999999999999}


#### Uncomment if you want to dump these files

In [23]:
write_file("en_2_hindi_IndicTrans.txt",en_2_hindi_IndicTrans)
write_file("hi_2_en_IndicTrans.txt",hi_2_en_IndicTrans)
write_file("mar_2_hindi_IndicTrans.txt",mar_2_hindi_IndicTrans)
write_file("hi_2_marathi_IndicTrans.txt",hi_2_marathi_IndicTrans)